In [ ]:
import glob
import json
import os
from multiprocessing import Pool, pool

import cv2
import numpy as np
import csv

import pandas as pd
from scipy.io import loadmat
import time
from matplotlib import pyplot as plt

In [ ]:
training_dir = '/Users/aloksaxena/Documents/aquabyteai/repos/cv_research/alok/notebooks/calibration/data/sf_test_images_4/all_images/'



In [ ]:
left_image_fs = sorted(glob.glob(os.path.join(training_dir, 'left*.jpg')))
right_image_fs = sorted(glob.glob(os.path.join(training_dir, 'right*.jpg')))

In [ ]:
im = cv2.imread(left_image_fs[0])
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
print(gray.shape[::-1])

In [ ]:
# define checkerboard dimensions and other optical properties
CHECKERBOARD_WIDTH = 9
CHECKERBOARD_HEIGHT = 6
IMAGE_PIXEL_WIDTH = 4096
IMAGE_PIXEL_HEIGHT = 3000
RESIZE_FACTOR = 4
CHECKERBOARD_SIDE_LENGTH = 0.0495

# create object points
obj_pts = np.zeros((CHECKERBOARD_HEIGHT*CHECKERBOARD_WIDTH,3), np.float32)
obj_pts[:,:2] = np.mgrid[0:CHECKERBOARD_WIDTH,0:CHECKERBOARD_HEIGHT].T.reshape(-1,2)
obj_pts = obj_pts * CHECKERBOARD_SIDE_LENGTH


# helper method for finding the checkerboard points in a given file path
def find_checkerboard_points(im):
    gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    resized_gray_im = cv2.resize(gray_im, (IMAGE_PIXEL_WIDTH // RESIZE_FACTOR, IMAGE_PIXEL_HEIGHT // RESIZE_FACTOR))
    ret, resized_corners = cv2.findChessboardCorners(resized_gray_im, (CHECKERBOARD_WIDTH, CHECKERBOARD_HEIGHT),None)
    unadjusted_corners, corners = None, None
    if ret:
        unadjusted_corners = resized_corners * RESIZE_FACTOR
        corners = cv2.cornerSubPix(gray, unadjusted_corners, (5, 5), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
    return ret, corners


# method for generating calibration matrix for each individual camera and then for the entire stereo system
def calibrate_stereo_camera(left_image_fs, right_image_fs):
    
    # first, perform calibration for each individual camera (left and right)
    left_intersection_pts = []
    right_intersection_pts = []
    for left_image_f, right_image_f in zip(left_image_fs, right_image_fs):
        left_im, right_im = cv2.imread(left_image_f), cv2.imread(right_image_f)
        left_ret, left_corners = find_checkerboard_points(left_im)
        right_ret, right_corners = find_checkerboard_points(right_im)
        
        # only consider well-behaved frame pairs (i.e. left and right image pairs where all checkerboard
        # points were found)
        if left_ret or right_ret:
            if left_ret and right_ret:
                print('Checkerboard points found for stereo image pair')
            else:
                print('Checkerboard points found for at least one image in stereo pair')
            left_intersection_pts.append(left_corners)
            right_intersection_pts.append(right_corners)
        else:
            print('Checkerboard points not found!')
    
    # get valid points
    valid_left_intersection_pts, valid_right_intersection_pts = [], []
    stereo_valid_left_intersection_pts, stereo_valid_right_intersection_pts = [], []
    for lip, rip in zip(left_intersection_pts, right_intersection_pts):
        if lip is not None:
            valid_left_intersection_pts.append(lip)
        if rip is not None:
            valid_right_intersection_pts.append(rip)
        if lip is not None and rip is not None:
            stereo_valid_left_intersection_pts.append(lip)
            stereo_valid_right_intersection_pts.append(rip)
        
    
    # generate rectification parameters for left camera
    valid_left_object_pts = [obj_pts] * len(valid_left_intersection_pts)
    left_ret, left_mtx, left_dist, left_rvecs, left_tvecs = \
        cv2.calibrateCamera(valid_left_object_pts, valid_left_intersection_pts, (IMAGE_PIXEL_WIDTH, IMAGE_PIXEL_HEIGHT), None, None)
    
    print('Left camera rectified!')
    
    # generate rectification parameters for right camera
    valid_right_object_pts = [obj_pts] * len(valid_right_intersection_pts)
    right_ret, right_mtx, right_dist, right_rvecs, right_tvecs = \
        cv2.calibrateCamera(valid_right_object_pts, valid_right_intersection_pts, (IMAGE_PIXEL_WIDTH, IMAGE_PIXEL_HEIGHT), None, None)
        
    print('Right camera rectified!')
    
    # generate rectification parameters for stereo system
    
    valid_stereo_object_pts = [obj_pts] * len(stereo_valid_left_intersection_pts)
    stereocalibration_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)
    stereocalibration_flags = cv2.CALIB_FIX_INTRINSIC
    stereocalibration_retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = cv2.stereoCalibrate(valid_stereo_object_pts, stereo_valid_left_intersection_pts, stereo_valid_right_intersection_pts, 
                       left_mtx, left_dist, right_mtx, right_dist, (IMAGE_PIXEL_WIDTH, IMAGE_PIXEL_HEIGHT),
                       criteria=stereocalibration_criteria, flags=stereocalibration_flags)
    
    print('Stereo system rectified')
    
    return stereocalibration_retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F, left_mtx


    
    

In [ ]:
stereocalibration_retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F, lm = calibrate_stereo_camera(left_image_fs, right_image_fs)

In [ ]:
cameraMatrix2

In [ ]:
newcameramtx1, roi1 = cv2.getOptimalNewCameraMatrix(cameraMatrix1, distCoeffs1, (4096, 3000), 1, (4096, 3000))
newcameramtx2, roi2 = cv2.getOptimalNewCameraMatrix(cameraMatrix2, distCoeffs2, (4096, 3000), 1, (4096, 3000))

In [ ]:
stereocalibration_retval

<h1> Assess Calibration Accuracy </h1>

In [ ]:
validation_dir = '/Users/aloksaxena/Documents/aquabyteai/repos/cv_research/alok/notebooks/calibration/data/sf_test_images_4/validation_images'
left_validation_image_fs = sorted(glob.glob(os.path.join(validation_dir, 'left*.jpg')))
right_validation_image_fs = sorted(glob.glob(os.path.join(validation_dir, 'right*.jpg')))

In [ ]:
world_checkerboard_points_list = []
for left_image_f, right_image_f in zip(left_validation_image_fs, right_validation_image_fs):
    left_im = cv2.imread(left_image_f)
    right_im = cv2.imread(right_image_f)
    left_rectified = cv2.undistort(left_im, cameraMatrix1, distCoeffs1, None, newcameramtx1)
    right_rectified = cv2.undistort(right_im, cameraMatrix2, distCoeffs2, None, newcameramtx2)
    
    left_ret, left_corners = find_checkerboard_points(left_im)
    right_ret, right_corners = find_checkerboard_points(right_im)
    
    
    if left_ret and right_ret:
        print('Checkerboard points found!')
        print(left_image_f)
        P1 = np.dot(cameraMatrix1, np.hstack([np.eye(3), np.array([[0], [0], [0]])]))
        P2 = np.dot(cameraMatrix2, np.hstack([R, T]))
        wcp = cv2.triangulatePoints(P1, P2, left_corners, right_corners).T
        world_checkerboard_points = wcp[:,:3] / wcp[:,3, None]
        world_checkerboard_points_list.append(world_checkerboard_points)
        
world_coordinate_matrix = np.empty([len(world_checkerboard_points_list), CHECKERBOARD_WIDTH*CHECKERBOARD_HEIGHT, 3])
for i, world_checkerboard_points in enumerate(world_checkerboard_points_list):
        world_coordinate_matrix[i, :] = world_checkerboard_points
    
    
    

In [ ]:
def euclidean_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5

def distance_between_positions(i, j):
    p1_ax_0_pos = i // CHECKERBOARD_WIDTH
    p1_ax_1_pos = i % CHECKERBOARD_WIDTH
    p2_ax_0_pos = j // CHECKERBOARD_WIDTH
    p2_ax_1_pos = j % CHECKERBOARD_WIDTH
    return CHECKERBOARD_SIDE_LENGTH * ((p1_ax_0_pos - p2_ax_0_pos)**2 + (p1_ax_1_pos - p2_ax_1_pos)**2)**0.5

In [ ]:
analysis_df = pd.DataFrame()
for idx in range(world_coordinate_matrix.shape[0]):
    for i in range(world_coordinate_matrix.shape[1]-1):
        for j in range(i+1, world_coordinate_matrix.shape[1]):
            p1 = world_coordinate_matrix[idx, i, :]
            p2 = world_coordinate_matrix[idx, j, :]
            predicted_distance = euclidean_distance(p1, p2)
            ground_truth_distance = distance_between_positions(i, j)
            row = {
                'idx': idx,
                'predicted_distance': predicted_distance,
                'ground_truth_distance': ground_truth_distance,
                'p1_x': p1[0],
                'p1_y': p1[1],
                'p1_z': p1[2],
                'p2_x': p2[0],
                'p2_y': p2[1],
                'p2_z': p2[2],
            }
            analysis_df = analysis_df.append(row, ignore_index=True)
    
analysis_df['deviation'] = analysis_df['predicted_distance'] - analysis_df['ground_truth_distance']
analysis_df['pct_deviation'] = analysis_df['deviation'] / analysis_df['ground_truth_distance']

In [ ]:
plt.hist(analysis_df.deviation)
plt.show()

In [ ]:
analysis_df['abs_deviation'] = analysis_df.deviation.abs()
analysis_df['yaw'] = (180 / np.pi) * np.arctan((analysis_df.p1_x - analysis_df.p2_x)/(analysis_df.p1_z - analysis_df.p2_z))
analysis_df['abs_yaw'] = analysis_df['yaw'].abs()

In [ ]:
plt.hist(analysis_df[(analysis_df.ground_truth_distance < 0.3) & (analysis_df.ground_truth_distance < 0.5)].deviation)
plt.show()


In [ ]:
plt.scatter(analysis_df.predicted_distance, analysis_df.ground_truth_distance)
plt.xlabel('Predicted Distance')
plt.plot([0, 0.5], [0, 0.5])
plt.show()

In [ ]:
analysis_df[analysis_df.ground_truth_distance > 0.45].sort_values('deviation')